In [ ]:
import asf_search as asf

In [ ]:
search_parameters = {
    "platform": ["S1"],
    "processingLevel": ["SLC"],
    "beamMode": ["IW"],
    "polarization": ["VV+VH"],
    "start": "2022-05-18",
    "end": "2022-12-01",
    "intersectsWith": "POLYGON((88.2733 26.8522,87.9897 26.8555,87.9767 26.8534,87.3067 26.7491,87.0011 26.8575,86.9938 26.8575,84.6891 27.6531,82.5953 28.1529,81.5007 28.7001,80.0969 29.0809,79.1861 29.615,78.5167 28.5766,78.7787 27.2874,78.6919 26.4344,78.6205 25.3707,78.5149 24.2368,78.5763 23.8111,78.8046 23.5573,79.2323 22.9953,79.3721 22.9435,79.5727 22.9467,80.1637 23.1417,80.5249 23.2157,80.9465 23.2017,81.7853 22.8307,81.8401 22.6401,82.0433 22.5511,82.8631 22.5995,82.8712 22.6001,82.8713 22.5989,83.0577 22.5431,83.2023 22.6805,83.1085 22.8209,83.3609 23.1285,83.9519 23.3525,84.1695 23.6465,84.4725 23.5153,84.6093 23.5967,84.8117 23.9187,84.9515 23.9705,85.0393 23.8681,85.2067 23.9395,85.6463 24.5699,85.8413 24.5311,85.9189 24.3827,86.1818 24.2755,86.1934 24.268,87.0539 23.8361,87.7399 23.5313,88.0203 23.2965,88.0786 23.0512,87.9981 22.2447,87.9641 21.9236,87.9821 21.7886,87.9975 21.6392,88.191 21.5641,88.2556 21.5643,88.2646 21.5644,88.8358 21.5667,90.1433 21.7556,90.6789 21.7432,91.1617 22.0114,91.5084 22.2822,91.7269 22.7256,91.9075 23.1854,92.0615 23.4098,92.1437 23.4281,92.4821 23.2537,92.7431 22.7425,92.7835 22.6984,92.8256 22.7276,92.8537 22.7658,93.5325 23.6863,93.7665 24.1093,93.8401 24.2885,94.1175 24.9533,94.1589 25.1475,94.4519 25.3565,94.5485 25.7317,94.7423 25.7553,94.8831 25.8907,95.0069 26.2627,94.9327 26.5995,95.0499 26.7365,95.5617 26.9655,96.6105 26.9397,96.7527 27.0529,96.7275 27.2537,96.7207 27.3849,96.7199 27.5913,96.6269 27.7277,96.4177 27.7913,95.5395 28.0577,95.0637 27.7723,93.5647 27.1297,92.0893 26.8205,91.9249 26.8092,90.7093 26.7253,89.7337 26.7253,88.8211 26.8469,88.4961 26.8513,88.2733 26.8522))"
}

In [ ]:
search_results = asf.geo_search(**search_parameters)
all_scenes = {result.geojson()['properties']['sceneName'] for result in search_results}
print(f'Found {len(all_scenes)} scenes')

In [ ]:
import hyp3_sdk as sdk

In [ ]:
project_name = 'HKHwatermaps'
hyp3 = sdk.HyP3('https://hyp3-watermap.asf.alaska.edu', prompt=True)

In [ ]:
processed_jobs = hyp3.find_jobs(name=project_name)
processed_scenes = {job.job_parameters['granules'][0] for job in processed_jobs}
print(f'We have already processed {len(processed_scenes)} scenes')

In [ ]:
scenes_to_process = all_scenes - processed_scenes
print(f'There are {len(scenes_to_process)} new scenes to process')

In [ ]:
import copy
from hyp3_sdk.util import chunk

In [ ]:
job_spec = {
    "job_type": "WATER_MAP",
    "job_name": project_name,
    "job_parameters": {
          "resolution": 30,
          "speckle_filter": True,
          "max_vv_threshold": -15.5,
          "max_vh_threshold": -23.0,
          "hand_threshold": 15.0,
          "hand_fraction": 0.8,
          "membership_threshold": 0.45,
          "include_flood_depth": True,
          "iterative_max": 15,
          "iterative_min": 0,
          "known_water_threshold": 30,
          "water_level_sigma": 3
    }
}

In [ ]:
prepared_jobs = []
for scene in scenes_to_process:
    job_dict = copy.deepcopy(job_spec)
    job_dict['job_parameters']['granules'] = [scene]
    prepared_jobs.append(job_dict)

In [ ]:
submitted_jobs = sdk.Batch()
for batch in chunk(prepared_jobs):
    submitted_jobs += hyp3.submit_prepared_jobs(batch)

In [ ]:
submitted_jobs = hyp3.watch(submitted_jobs)
